In [1]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import json

df_apps = pd.read_csv(r"../data/Google-Playstore-Dataset-Clean.csv")
print(df_apps)

                      App Name         Category Installs  Size  \
0                      Gakondo        Adventure      10+   10M   
1          Ampere Battery Info            Tools   5,000+  2.9M   
2                       Vibook     Productivity      50+  3.7M   
3                       IMOCCI           Social      50+   46M   
4        The Everyday Calendar        Lifestyle     500+   16M   
...                        ...              ...      ...   ...   
1057516       Floral Wallpaper  Personalization   1,000+   29M   
1057517      Engineers Careers         Business     100+   21M   
1057518             ORU Online        Education     100+   44M   
1057519         Data Structure        Education     100+   29M   
1057520            Devi Suktam    Music & Audio   1,000+   10M   

        Minimum Android      Released  Last Updated Content Rating  Rating  \
0            7.1 and up  Feb 26, 2020  Feb 26, 2020       Everyone     0.0   
1            5.0 and up  May 21, 2020  May 06, 2021

In [56]:
def create_database(db_name):
    with open('db_config.json', 'r') as config_file:
        config = json.load(config_file)

    conn = psycopg2.connect(
        host=config['host'],
        user=config['user'],
        password=config['password']
    )
    conn.autocommit = True

    cursor = conn.cursor()

    cursor.execute(f"DROP DATABASE IF EXISTS {db_name};")
    cursor.execute(f"CREATE DATABASE {db_name};")

    cursor.close()
    conn.close()
    print(f"the database '{db_name}' has been sucessfully created.")
    
create_database("playstore_apps")

ObjectInUse: la base de datos «playstore_apps» está siendo utilizada por otros usuarios
DETAIL:  Hay otras 3 sesiones usando la base de datos.


In [82]:
def connect_to_db():
    db_conn = None
    try:
        with open('db_config.json', 'r') as config_file:
            db_settings = json.load(config_file)

        db_conn = psycopg2.connect(
            host='localhost',
            user=db_settings['user'],
            password=db_settings['password'],
            dbname=db_settings['database']
        )
        print('Connection to the database was successful')
    except psycopg2.DatabaseError as db_error:
        print('Failed to connect to the database:', db_error)
    return db_conn

connect_to_db()

Connection to the database was successful


<connection object at 0x0000012359506460; dsn: 'user=postgres password=xxx dbname=playstore_apps host=localhost', closed: 0>

In [85]:
def setup_apps_table():
    table_creation_sql = '''
        CREATE TABLE IF NOT EXISTS apps (
            id SERIAL PRIMARY KEY,
            app_name VARCHAR(255) NOT NULL,
            category VARCHAR(255) NOT NULL,
            installs VARCHAR(255) NOT NULL,
            size VARCHAR(255) NOT NULL,
            minimum_android VARCHAR(255) NOT NULL,
            released DATE NOT NULL,
            last_updated DATE NOT NULL,
            content_rating VARCHAR(255) NOT NULL,
            rating FLOAT NOT NULL,
            minimum_installs INT NOT NULL,
            maximum_installs INT NOT NULL
        );
    '''
    db_connection = None
    try:
        db_connection = connect_to_db()
        db_cursor = db_connection.cursor()
        db_cursor.execute(table_creation_sql)
        db_cursor.close()
        db_connection.commit()
        print('The "apps" table has been successfully created.')
    except (Exception, psycopg2.DatabaseError) as db_error:
        print('Error while creating the table:', db_error)
    finally:
        if db_connection is not None:
            db_connection.close()

setup_apps_table()

Connection to the database was successful
The "apps" table has been successfully created.


In [87]:
def insertar_datos(df):
    conn = connect_to_db()
    if conn is None:
        print("No se pudo establecer la conexión con la base de datos.")
        return

    cursor = conn.cursor()
    query = """
    INSERT INTO apps (id, app_name, category, installs, size, minimum_android, released, last_updated, content_rating, rating, minimum_installs, maximum_installs)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        for index, row in df.iterrows():
            data = (row["ID"], row["App Name"], row["Category"], row["Installs"], row["Size"],
                    row["Minimum Android"], row["Released"], row["Last Updated"], row["Content Rating"], row["Rating"], row["Minimum Installs"], row["Maximum Installs"])

            cursor.execute(query, data)

        conn.commit()
        print("Datos insertados exitosamente")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error al insertar los datos:", error)
        conn.rollback()
    finally:
        cursor.close()
        conn.close()

insertar_datos(nuevo_df_apps)

Connection to the database was successful
Datos insertados exitosamente
